In [1]:
#####Note:run in python2######
# same initialization of weights because same random seed used

import math
import numpy as np
import sys
from PIL import Image
import matplotlib.pyplot as plt 

#for shuffling data
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

In [2]:
#########
#MLP

# initialisation of the weights he_normal
def weights(noHiddenLayers,sizeOfLayers):

    W=[]
    b=[]

    for i in range(0,noHiddenLayers+1):
        
        W.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+sizeOfLayers[i])),
                                   (sizeOfLayers[i+1],sizeOfLayers[i])) )
        b.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+1)),
                                   (sizeOfLayers[i+1],1)) )

    W=np.array(W)
    b=np.array(b)
    
    return W,b

#mlp forward pass
#layer
def layer(w,x,b):
    out = np.matmul(w,x)+b
    return out

def apply_activationMLP(Activation_function,inp):
    
    #activation functions
    if Activation_function == 'relu':
        return np.where(inp<0,0,inp)
    elif Activation_function == "tanh":
        return np.tanh(inp)
    elif Activation_function == "sigmoid":
        return 1.0/(1+np.exp(-1.0*inp))
    elif Activation_function == "softmax":
        return (1.0/(np.sum(np.exp(inp),axis=0)))*(np.exp(inp))

#forward path
def forward_path(noHiddenLayers,X,W,b,Actfnvect):

    out=[]
    
    z=apply_activationMLP(Actfnvect[0],np.array(layer(W[0],X,b[0])))
    out.append(np.array(z))

    for i in range(1,noHiddenLayers):
        z=apply_activationMLP(Actfnvect[i],np.array(layer(W[i],out[i-1],b[i])))
        out.append(np.array(z))

    if noHiddenLayers > 0:
        z=apply_activationMLP(Actfnvect[-1],np.array(layer(W[-1],out[-1],b[-1])))
        out.append(np.array(z))

    y_pred = out[-1]

    return np.array(out),np.array(y_pred)


In [3]:
#only to import data
from keras import backend as K
import keras
from keras.datasets import mnist
 
    
################################

# #import data
iris_data = load_iris() # load the iris dataset

x = iris_data.data
y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
# Split the data for training and testing
x_train , x_test, y_train , y_test = train_test_split(x, y, test_size=0.20)

print(x_train.shape)
print(y_train.shape)

Using TensorFlow backend.


(120, 4)
(120, 3)


In [4]:
# run MLP algorithm

x_train = np.moveaxis(x_train,0,-1)
y_train = np.moveaxis(y_train,0,-1)
x_test = np.moveaxis(x_test,0,-1)
y_test = np.moveaxis(y_test,0,-1)

print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(4, 120)
(3, 120)
(4, 30)
(3, 30)


In [5]:
#derivative of relu
def der_relu(x):
    return np.where(x == 0,0,1)

# backpropation
def backprop(y,y_true,z,T,u,M,x):
    
    dy = (y-y_true)
    
    # layer 3
    dT  = np.matmul(dy,z.T)
    db3 = np.sum(dy,axis=1).reshape(-1,1)
    
    # layer 2
    s   = np.matmul(T.T,dy)
    s   = s*der_relu(z)
    dM  = np.matmul(s,u.T)
    db2 = np.sum(s,axis=1).reshape(-1,1)
    
    # layer 1
    sm  = np.matmul(M.T,s)
    sm  = sm*der_relu(u)
    dW  = np.matmul(sm,x.T)
    db1 = np.sum(sm,axis=1).reshape(-1,1)
    
    return dW,dM,dT,db1,db2,db3

In [6]:
###
# Training parameters
num_classes = 3
epochs = 600
learning_rate=1e-4
batch_size = 10

################################
#MLP PARAMETERS
noHiddenLayers=2

#also includes the input vector dimension and output vector dimension
sizeOfLayers=[x_train.shape[0],10,10,num_classes]

sizeofOutput=num_classes

Actfnvect = ['relu','relu','softmax']

In [7]:
W,b = weights(noHiddenLayers,sizeOfLayers)


In [8]:
# (a)
# stochastic gradient descent

# updating the weights



for i in range(epochs):
    loss=0
    for j in np.arange(0,x_train.shape[1],batch_size):       
        dW = np.zeros(W[0].shape)
        dM = np.zeros(W[1].shape)
        dT = np.zeros(W[2].shape)
        db1 = np.zeros(b[0].shape)
        db2 = np.zeros(b[1].shape)
        db3 = np.zeros(b[2].shape)
        for k in range(0,batch_size):
            # forward pass
            x = x_train[:,j+k].reshape(-1,1)
            y = y_train[:,j+k].reshape(-1,1)
            out,y_pred=forward_path(noHiddenLayers,x,W,b,Actfnvect)
            # backpropagation
            dWtemp,dMtemp,dTtemp,db1temp,db2temp,db3temp=backprop(y_pred,y,out[1],W[-1],out[0],W[1],x)
            
            dW=dW+dWtemp
            db1=db1+db1temp
            
            dM=dM+dMtemp
            db2=db2+db2temp
            
            dT=dT+dTtemp
            db3=db3+db3temp
        
            # calculate the loss
            loss = loss + (-1.0*np.sum(y*np.log(y_pred)))
            
        # Updating the weights SGD
        b[0]=b[0]-learning_rate*db1*(1.0/batch_size)
        W[0]=W[0]-learning_rate*dW*(1.0/batch_size)

        b[1]=b[1]-learning_rate*db2*(1.0/batch_size)
        W[1]=W[1]-learning_rate*dM*(1.0/batch_size)

        b[2]=b[2]-learning_rate*db3*(1.0/batch_size)
        W[2]=W[2]-learning_rate*dT*(1.0/batch_size)
    
    #print the loss in each epoch
    print('Epoch:'+str(i)+'         Loss:'+str(loss))

Epoch:0         Loss:260.7953073984197
Epoch:1         Loss:258.01363975313905
Epoch:2         Loss:255.32596057898502
Epoch:3         Loss:252.7275082881799
Epoch:4         Loss:250.2248580726914
Epoch:5         Loss:247.8281201790449
Epoch:6         Loss:245.5305658303528
Epoch:7         Loss:243.30297692661122
Epoch:8         Loss:241.1411458462219
Epoch:9         Loss:239.04453378140147
Epoch:10         Loss:237.01014632191823
Epoch:11         Loss:235.0351740009979
Epoch:12         Loss:233.11539841110127
Epoch:13         Loss:231.2430049278045
Epoch:14         Loss:229.41765984135694
Epoch:15         Loss:227.63360828136518
Epoch:16         Loss:225.8859418455903
Epoch:17         Loss:224.18570758347084
Epoch:18         Loss:222.52843521638817
Epoch:19         Loss:220.9181021023612
Epoch:20         Loss:219.3500532355199
Epoch:21         Loss:217.82379163969904
Epoch:22         Loss:216.3349153489726
Epoch:23         Loss:214.8822412924554
Epoch:24         Loss:213.4645414172800

Epoch:209         Loss:126.51824410361986
Epoch:210         Loss:126.29953123944081
Epoch:211         Loss:126.08133283696182
Epoch:212         Loss:125.86362537969391
Epoch:213         Loss:125.64603315746723
Epoch:214         Loss:125.42774051896332
Epoch:215         Loss:125.21014246001947
Epoch:216         Loss:124.99286890360418
Epoch:217         Loss:124.77591953551581
Epoch:218         Loss:124.55954853048536
Epoch:219         Loss:124.34319128206346
Epoch:220         Loss:124.12484407453371
Epoch:221         Loss:123.90658924128913
Epoch:222         Loss:123.68907402487355
Epoch:223         Loss:123.47226611726921
Epoch:224         Loss:123.25567552079393
Epoch:225         Loss:123.03790369529
Epoch:226         Loss:122.8170744519897
Epoch:227         Loss:122.59579038396791
Epoch:228         Loss:122.37475857280432
Epoch:229         Loss:122.15398237690849
Epoch:230         Loss:121.93076376974246
Epoch:231         Loss:121.7026160267842
Epoch:232         Loss:121.469908239914

Epoch:415         Loss:83.92050868525452
Epoch:416         Loss:83.81990038581753
Epoch:417         Loss:83.71973621967413
Epoch:418         Loss:83.62000473382305
Epoch:419         Loss:83.52112520977956
Epoch:420         Loss:83.42386076788668
Epoch:421         Loss:83.32703666010246
Epoch:422         Loss:83.23064099601709
Epoch:423         Loss:83.13526506527984
Epoch:424         Loss:83.04160306292079
Epoch:425         Loss:82.94917201835467
Epoch:426         Loss:82.85721060359107
Epoch:427         Loss:82.76570440820286
Epoch:428         Loss:82.6746396866021
Epoch:429         Loss:82.58400332521161
Epoch:430         Loss:82.49378281125891
Epoch:431         Loss:82.4043422662992
Epoch:432         Loss:82.31613128552004
Epoch:433         Loss:82.22834701424988
Epoch:434         Loss:82.14097640380889
Epoch:435         Loss:82.05401483244493
Epoch:436         Loss:81.96738871086096
Epoch:437         Loss:81.88109219166141
Epoch:438         Loss:81.79548057413764
Epoch:439         

In [9]:
out,y_pred=forward_path(noHiddenLayers,x_test,W,b,Actfnvect)

In [10]:
def predict(y):
    return np.argmax(y)

yvect=[]
y_trurevect=[]

for i in range(0,x_test.shape[1]):
    yvect.append(predict(y_pred[:,i]))
    y_trurevect.append(predict(y_test[:,i]))

# find accuracy
from sklearn.metrics import accuracy_score
#predicting test accuracy
print(accuracy_score(y_trurevect, yvect))

1.0


In [11]:
# to see the output vs true values

print y_trurevect
print yvect

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
